## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

In [1]:
import os
import sys
import random
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.model_selection import train_test_split

In [2]:
# Global variables

# Vocabulary
vocab_size = 3000
context_size = 5
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 32
epochs = 10
lr = 0.01

# Model
embedding_dim = 64
hidden_dim = 128

## Faz download e carrega o dataset

In [3]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

--2024-03-17 20:19:43--  https://www.gutenberg.org/ebooks/67724.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/67724/pg67724.txt [following]
--2024-03-17 20:19:44--  http://www.gutenberg.org/cache/epub/67724/pg67724.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/67724/pg67724.txt [following]
--2024-03-17 20:19:44--  https://www.gutenberg.org/cache/epub/67724/pg67724.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 372908 (364K) [text/plain]
Saving to: '67724.txt.utf-8'

     0K .......... .......... .......... .......... .......... 13%  172K 2s
    50K

In [4]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [5]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [6]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

--Confio em Deos, meu pai, que vossos receios serão infundados; mas se elle nos quizesse submetter a tal provança, o unico lugar que compete a vosso filho e herdeiro de vosso nome é nesta casa ameaçada, ao vosso lado, para defender-vos e partilhar a vossa sorte, qualquer que ella seja.

--Aqui me tendes! repetio o cavalheiro. Dizei o que quereis de D. Antonio de Mariz, e dizei-o claro e breve. Se fôr de justiça, sereis satisfeitos; se fôr uma falta, tereis a punição que merecerdes.

--É impossivel!... gritou elle. Isso que dizeis é falso. Não ha homem que o fizesse.

O indio no seu fanatismo não comprehendia que houvesse uma razão capaz de sacrificar a vida de Cecilia, que para elle era sagrada.

Deixando Alvaro, a intenção do indio era atalhar os aventureiros, espera-los junto á cerca; e quando elles se separassem para entrar a um e um, mata-los.

Number of paragraphs: 4892


4892

## Análise do dataset

In [7]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [8]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [9]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 3000


In [10]:
#pattern = r'\w+'

def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 0, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [11]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [12]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 44547
Validation dataset samples: 12257


In [13]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[   4,   33,   71,    6, 2254],
        [  17,  427,    2,  136, 1297],
        [  30,    1,  279,    2, 1655],
        [1348, 2992,    4,    5, 2102],
        [  19,  202,    2,  291,   21],
        [   2,    7,  858,   38,   40],
        [   4, 1002,  106, 1224,    6],
        [  24,   19,  523,    6,  277],
        [  79, 2264,   34,  127,   13],
        [1658,    9,   33,    5,  192],
        [   6,   64,    9,   33,  272],
        [   3, 2459,    9,   12,   52],
        [ 621,    7, 1993,    4, 1563],
        [   1,    1,    1,    1,    1],
        [  12,    9,   12,    5,  247],
        [  52,    9,   39,  114,   12],
        [   6,   75,  165,   95, 1317],
        [1414,    2,   13,  773,    2],
        [1265,  475,   30,   61,  325],
        [ 180,    1,   30,    1,   47],
        [   2,   50,   54,   16,   10],
        [  14,  149,    5, 1125,  767],
        [   9,    5,  102,  573,  579],
        [  35, 1060,   56,    9,    7],
        [   7, 1226,   27,  227,   22],

## Model

In [14]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.linear2 = nn.Linear(h, vocab_size+1)
        #self.relu = torch.nn.ReLU()
        self.tanh = torch.nn.Tanh()

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        #out = self.relu(out)
        out = self.tanh(out)
        # Second layer
        out = self.linear2(out)
        return out

In [15]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)

In [16]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

torch.Size([32, 5])
torch.Size([32])


In [17]:
output = model(input)

In [18]:
output.argmax(dim=1)

tensor([ 725, 2406, 2240, 2987, 1755, 2552,  178, 2822, 2837,  974,  430, 2014,
        2557,  291, 1038, 2870, 2712, 1895, 1777, 1304, 2482, 2606, 1816, 2397,
        2830,  618,  274,  803, 1427,  684,  652, 2701])

In [19]:
target

tensor([1746, 2075,   70,    2,  864,   48,   92,    9, 1023,   29,  320,   17,
           4,    1,    1,  382,    3,   48,   23,    4,   27,  158,   12,    5,
        2518,  110,    8,    1, 1257,   10,    1, 1316])

## Training

In [20]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [21]:
# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr)

model.to(device)

BengioModel(
  (embeddings): Embedding(3001, 64)
  (linear1): Linear(in_features=320, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=3001, bias=True)
  (tanh): Tanh()
)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exemplo de uso:
total_params = count_parameters(model)
print(f'O modelo tem um total de {total_params:,} parâmetros.')

O modelo tem um total de 620,281 parâmetros.


In [23]:
# Initial Perplexity and Loss
# Before training
model.eval()

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

loss /= len(train_loader)
perp = torch.exp(torch.tensor(loss))

print(f'Initial Loss: {loss:.4f}')
print(f'Initial Perplexity: {perp:.4f}')

Initial Loss: 8.0380
Initial Perplexity: 3096.3850


In [24]:
# Training Loop
model.train()
for epoch in range(epochs):

  epoch_start = time.time()
  # Metrics
  epoch_loss = 0
  epoch_correct = 0
  epoch_samples = 0

  for inputs, targets in train_loader:
        inputs = inputs.to(device)  # Move input data to the device
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss
        epoch_loss += loss.item()

        # Predicted
        _, predicted = torch.max(outputs, 1)
        epoch_correct += (predicted == targets).sum().item()
        epoch_samples += targets.size(0)

  # Calculate average loss and accuracy for epoch
  avg_loss = epoch_loss / len(train_loader)
  acc = epoch_correct / epoch_samples

  # Perplexity
  perp = torch.exp(torch.tensor(avg_loss))

  epoch_end = time.time()
  epoch_time = epoch_end - epoch_start
  # Print epoch statistics
  print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


Epoch [1/10], Time:5.30, Loss: 6.3488, Accuracy: 0.11%, Perplexity: 571.7896
Epoch [2/10], Time:5.51, Loss: 5.5139, Accuracy: 0.15%, Perplexity: 248.1158
Epoch [3/10], Time:5.14, Loss: 5.0005, Accuracy: 0.18%, Perplexity: 148.4801
Epoch [4/10], Time:5.14, Loss: 4.6462, Accuracy: 0.21%, Perplexity: 104.1842
Epoch [5/10], Time:5.14, Loss: 4.3717, Accuracy: 0.24%, Perplexity: 79.1784
Epoch [6/10], Time:5.35, Loss: 4.1872, Accuracy: 0.26%, Perplexity: 65.8386
Epoch [7/10], Time:5.90, Loss: 4.0356, Accuracy: 0.28%, Perplexity: 56.5778
Epoch [8/10], Time:5.74, Loss: 3.9409, Accuracy: 0.29%, Perplexity: 51.4654
Epoch [9/10], Time:5.74, Loss: 3.8735, Accuracy: 0.30%, Perplexity: 48.1094
Epoch [10/10], Time:5.88, Loss: 3.7960, Accuracy: 0.31%, Perplexity: 44.5238


## Avaliação

In [25]:
model.eval()

loss_sum = 0
total_sum = 0
correct_sum = 0
eval_round = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)        
        loss_sum += loss

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        total_sum += targets.size(0)
        correct_sum += (predicted == targets).sum().item()
        eval_round += 1

# Calculate accuracy
acc = 100 * correct_sum/total_sum

# Calculate average perplexity
average_loss = loss_sum / eval_round
average_perplexity = torch.exp(average_loss)

print(f'Test Accuracy: {acc:.2f}%')
print(f'Average Loss: {average_loss:.2f}')
print(f'Average Perplexity: {average_perplexity:.2f}')

Test Accuracy: 19.30%
Average Loss: 9.58
Average Perplexity: 14409.77


## Exemplo de uso

In [31]:
# Código adaptado da implementação do Cesar Bastos
from colorama import Fore, Style

text = cleaned_paragraphs
model.to(device)
def generate_text(model, vocab, text, max_length, context_size):
    words = []
    # Ensure there are enough words for at least one sequence
    while len(words) < context_size:
        random_number = random.randint(1, 4891)
        words = encode_sentence(text[random_number], vocab)
        if not words:
            words = []
            continue  # Skip if the sentence cannot be encoded
        words = words[:context_size]
        #print(words)
        if any(token == 0 for token in words):
            words = []
            continue  # Skip if any token is zero (assuming 0 is a special token)
        context = words

    print(f"Frase: {cleaned_paragraphs[random_number]}")
    print(words)

    for _ in range(max_length):
        words_tensor = torch.tensor(context[-context_size:], dtype=torch.long).unsqueeze(0).to(device)
        logits = model(words_tensor)
        probs = F.softmax(logits, dim=1)
        next_token = torch.multinomial(probs, num_samples=1)
        context.append(next_token.item())
        print(context)
    frase = []
    for i in context: ##Agradecimentos a Ramon Abilio
        word = next((word for word, code in vocab.items() if code == i), "<UNKNOWN>")
        frase.append(word)

    print(f"{Fore.BLUE}{frase[:context_size]}{Style.RESET_ALL} {Fore.RED}{frase[-max_length:]}{Style.RESET_ALL} ")


max_length= 10
generate_text(model, vocab, text, max_length, context_size)

Frase: --Já que o quereis, força é satisfazer-vos. Noto que a ordem de D. Antonio, e o italiano carregou nesta palavra, manda-vos estar no _Paquequer_ um pouco antes de seis horas, a tempo de ouvir a prece.
[72, 4, 5, 463, 2]
[72, 4, 5, 463, 2, 2195]
[72, 4, 5, 463, 2, 2195, 64]
[72, 4, 5, 463, 2, 2195, 64, 12]
[72, 4, 5, 463, 2, 2195, 64, 12, 69]
[72, 4, 5, 463, 2, 2195, 64, 12, 69, 115]
[72, 4, 5, 463, 2, 2195, 64, 12, 69, 115, 252]
[72, 4, 5, 463, 2, 2195, 64, 12, 69, 115, 252, 628]
[72, 4, 5, 463, 2, 2195, 64, 12, 69, 115, 252, 628, 3]
[72, 4, 5, 463, 2, 2195, 64, 12, 69, 115, 252, 628, 3, 17]
[72, 4, 5, 463, 2, 2195, 64, 12, 69, 115, 252, 628, 3, 17, 685]
['já', 'que', 'o', 'quereis', ','] ['encaminhou', 'isabel', 'não', 'estava', 'podia', 'si', 'tornou', 'a', 'sua', 'segunda'] 
